<a href="https://colab.research.google.com/github/freddiebermingham/Testing-Computational-Hypotheses-of-Word-and-Face-Recognition/blob/main/Split_conv4_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid
import os

import numpy as np
import pandas as pd
import seaborn as sn
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
transform = transforms.ToTensor()

In [ ]:
train_data = datasets.ImageFolder(('/content/gdrive/MyDrive/WF/train'), transform=transform)
test_data = datasets.ImageFolder(('/content/gdrive/MyDrive/WF/test'), transform=transform)

torch.manual_seed(42)
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)
test_loader = DataLoader(test_data, batch_size=10, shuffle=True)

class_names = train_data.classes
labels = train_data.targets

print(class_names)
print(f'Training images available: {len(train_data)}')
print(f'Testing images available:  {len(test_data)}')

['Aidan_Turner', 'Ashley_Rickards', 'Christina_Hendricks', 'Christine_Lahti', 'Cillian_Murphy', 'Dawn_French', 'Jacqueline_MacInnes_Wood', 'Kari_Matchett', 'Martha_Plimpton', 'Rowan_Atkinson', 'cash ', 'cats', 'fact', 'foot', 'heat', 'holt', 'lots', 'safe', 'soft', 'tell']
Training images available: 9004
Testing images available:  1600


In [ ]:
train_data.classes

['Aidan_Turner',
 'Ashley_Rickards',
 'Christina_Hendricks',
 'Christine_Lahti',
 'Cillian_Murphy',
 'Dawn_French',
 'Jacqueline_MacInnes_Wood',
 'Kari_Matchett',
 'Martha_Plimpton',
 'Rowan_Atkinson',
 'cash ',
 'cats',
 'fact',
 'foot',
 'heat',
 'holt',
 'lots',
 'safe',
 'soft',
 'tell']

In [ ]:
torch.manual_seed(500)  

train_loader = DataLoader(train_data, batch_size=10, shuffle=True)
test_loader = DataLoader(test_data, batch_size=10, shuffle=False)

In [ ]:
np.set_printoptions(formatter=dict(int=lambda x: f'{x:5}'))

for images,labels in train_loader: 
    break

print('Label:', labels.numpy())
print('Class: ', *np.array([class_names[i] for i in labels]))


im = make_grid(images, nrow=5) 
plt.figure(figsize=(10,4))
plt.imshow(np.transpose(im.numpy(), (1, 2, 0)));

In [ ]:
class ConvolutionalNetwork(nn.Sequential):
    def __init__(self):
        super(ConvolutionalNetwork, self).__init__()
        out1 = 32
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 24, 3, 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(24, 12, 3, 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(12, 24, 3, 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(24, out1, 3, 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2))
            
            
        self.layer_m = nn.Sequential(
            nn.Conv2d(out1, 32, 3, 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2))
        
        self.layer_c = nn.Sequential(
            nn.Conv2d(out1, 32, 3, 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2))
        
        self.fc_m = nn.Sequential(
            nn.Linear(5*5*32, 240),
            nn.ReLU(),
            nn.Linear(240,20))
        
        self.fc_c = nn.Sequential(
            nn.Linear(5*5*32, 240),
            nn.ReLU(),
            nn.Linear(240,20))

    def forward(self, X):
        X1 = self.layer1(X)
        Xm = self.layer_m(X1)
        Xc = self.layer_c(X1)   
        
        Xm = Xm.view(-1, 5*5*32)
        Xc = Xc.view(-1, 5*5*32)
        
        Xm = self.fc_m(Xm)
        Xc = self.fc_c(Xc)
        
        return F.log_softmax(Xm, dim=1)
        return F.log_softmax(Xc, dim=1)

In [ ]:
torch.manual_seed(500)
model = ConvolutionalNetwork()
model

In [ ]:
def count_parameters(model):
    params = [p.numel() for p in model.parameters() if p.requires_grad]
    for item in params:
        print(f'{item:>6}')
    print(f'______\n{sum(params):>6}')

In [ ]:
count_parameters(model)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
import time
start_time = time.time()

epochs = 10
train_losses = []
test_losses = []
train_correct = []
test_correct = []

for i in range(epochs):
    trn_corr = 0
    tst_corr = 0
    
    # training batches
    for b, (X_train, y_train) in enumerate(train_loader):
        b+=1
        
        y_pred = model(X_train)
        loss = criterion(y_pred, y_train)
 
        # Tally number of correct predictions
        predicted = torch.max(y_pred.data, 1)[1]
        batch_corr = (predicted == y_train).sum()
        trn_corr += batch_corr
        
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # results
        if b%100 == 0:
            print(f'epoch: {i:2}  batch: {b:4} [{10*b:6}/9000]  loss: {loss.item():10.8f}  \
accuracy: {trn_corr.item()*100/(10*b):7.3f}%  Duration: {time.time() - start_time:.0f} seconds')
        
    train_losses.append(loss)
    train_correct.append(trn_corr)
        
    # testing batches
    with torch.no_grad():
        for b, (X_test, y_test) in enumerate(test_loader):

            
            y_val = model(X_test)

            # Tally number of correct predictions
            predicted = torch.max(y_val.data, 1)[1] 
            tst_corr += (predicted == y_test).sum()
            
    loss = criterion(y_val, y_test)
    test_losses.append(loss)
    test_correct.append(tst_corr)
        
print(f'\nDuration: {time.time() - start_time:.0f} seconds')

epoch:  0  batch:  100 [  1000/9000]  loss: 2.18429327  accuracy:  10.900%  Duration: 228 seconds
epoch:  0  batch:  200 [  2000/9000]  loss: 1.31744087  accuracy:  26.600%  Duration: 448 seconds
epoch:  0  batch:  300 [  3000/9000]  loss: 0.99146307  accuracy:  38.300%  Duration: 662 seconds
epoch:  0  batch:  400 [  4000/9000]  loss: 1.08296978  accuracy:  45.425%  Duration: 862 seconds
epoch:  0  batch:  500 [  5000/9000]  loss: 0.89173096  accuracy:  50.960%  Duration: 1051 seconds
epoch:  0  batch:  600 [  6000/9000]  loss: 1.72418153  accuracy:  54.633%  Duration: 1293 seconds
epoch:  0  batch:  700 [  7000/9000]  loss: 0.35340646  accuracy:  57.771%  Duration: 1464 seconds
epoch:  0  batch:  800 [  8000/9000]  loss: 0.30336401  accuracy:  60.050%  Duration: 1634 seconds
epoch:  0  batch:  900 [  9000/9000]  loss: 1.05529392  accuracy:  61.867%  Duration: 1803 seconds
epoch:  1  batch:  100 [  1000/9000]  loss: 0.79795516  accuracy:  81.100%  Duration: 2083 seconds
epoch:  1  bat

In [ ]:
print(test_correct)
print()
print(f'Test accuracy: {test_correct[-1].item()*100/1600:.3f}%')

[tensor(1299), tensor(1427), tensor(1492), tensor(1471), tensor(1505), tensor(1537), tensor(1516), tensor(1515), tensor(1491), tensor(1511)]

Test accuracy: 94.438%


In [ ]:
# Create a loader for the entire the test set
test_load_all = DataLoader(test_data, batch_size=800, shuffle=False)

with torch.no_grad():
    correct = 0
    for X_test, y_test in test_load_all:
        y_val = model(X_test)
        predicted = torch.max(y_val,1)[1]
        correct += (predicted == y_test).sum()

arr = confusion_matrix(y_test.view(-1), predicted.view(-1))
df_cm = pd.DataFrame(arr, class_names, class_names)
plt.figure(figsize = (9,6))
sn.heatmap(df_cm, annot=True, fmt="d", cmap='BuGn')
plt.xlabel("prediction")
plt.ylabel("label (ground truth)")
plt.show();

ValueError: ignored

In [ ]:
plt.plot([t/90 for t in train_correct], label='training accuracy')
plt.plot([t/16 for t in test_correct], label='validation accuracy')
plt.title('Accuracy at the end of each epoch')
plt.legend();

In [ ]:
torch.save(model.state_dict(), '/content/gdrive/MyDrive/SPLIT4:6.pt')